# Send and receive a pulse 

This notebook is based on the [send-receive-pulse demo](../qick_demos/00_Send_receive_pulse.ipynb).

In [ ]:
# You should be on the ml branch of https://github.com/GiuseppeDiGuglielmo/qick
!git branch

In [ ]:
!echo -n "git rev:  "; git rev-parse --short HEAD
!echo -n "git date: "; git log -1 --format=%cd

In [ ]:
import subprocess
branch_name = subprocess.getoutput("git rev-parse --abbrev-ref HEAD")
print(branch_name)

In [ ]:
# Import the QICK drivers and auxiliary libraries
from qick import *
%pylab inline

In [ ]:
# Current directory
!pwd

In [ ]:
# List custom bitstreams
!ls -l 216/$branch_name/'00000000'

In [ ]:
!ls -l 216/$branch_name/'20240501'

In [ ]:
# Choose custom bistreams

DATASET_DATE = '00000000'
#DATASET_DATE = '20240501'

# QICK
#CUSTOM_BIT = '216/{}/qick_216_orig.bit'.format(branch_name)
#HAS_NN = 0

# QICK + ILAs + syn/imp optimizations
# CUSTOM_BIT = '216/ml-dev/qick_216_ila.bit'
# HAS_NN = 0

# QICK + NN + syn/imp optimizations
#CUSTOM_BIT = '216/{}/{}/qick_216_nn_285_385.bit'.format(branch_name, DATASET_DATE)
CUSTOM_BIT = '216/{}/{}/qick_216_nn_150_550_updated.bit'.format(branch_name, DATASET_DATE)
#CUSTOM_BIT = '216/{}/{}/qick_216_nn_0_770.bit'.format(branch_name, DATASET_DATE)
HAS_NN = 1

# Configure channels
GEN_CH = 0
RO_CH = 0

In [ ]:
# Load bitstream with custom overlay
if not 'CUSTOM_BIT' in locals():
    soc = QickSoc()
else:
    import os
    # Normalize path
    CUSTOM_BIT_FULL_PATH = os.path.normpath(os.getcwd() + '/' + CUSTOM_BIT)
    print('Custom bitsream:', CUSTOM_BIT_FULL_PATH)
    
    soc = QickSoc(bitfile=CUSTOM_BIT_FULL_PATH)

soccfg = soc
print(soccfg)

In [ ]:
print('Loaded bitstream:', soccfg.bitfile_name)

### Hardware Configuration

<!--generator channel 6 : DAC 229 CH3  <-> Readout channel 0 : ADC 224 CH0-->

In [ ]:
print('Generator channel: {}'.format(GEN_CH))
print('Readout channel  : {}'.format(RO_CH))

In [ ]:
class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for ch in cfg["ro_chs"]:
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
        phase = self.deg2reg(cfg["res_phase"], gen_ch=res_ch)
        gain = cfg["pulse_gain"]
        self.default_pulse_registers(ch=res_ch, freq=freq, phase=phase, gain=gain)

        style=self.cfg["pulse_style"]

        if style in ["flat_top","arb"]:
            sigma = cfg["sigma"]
            self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)
            
        if style == "const":
            self.set_pulse_registers(ch=res_ch, style=style, length=cfg["length"])
        elif style == "flat_top":
            # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure", length=cfg["length"])
        elif style == "arb":
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure")
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # fire the pulse
        # trigger all declared ADCs
        # pulse PMOD0_0 for a scope trigger
        # pause the tProc until readout is done
        # increment the time counter to give some time before the next measurement
        # (the syncdelay also lets the tProc get back ahead of the clock)
        self.measure(pulse_ch=self.cfg["res_ch"], 
                     adcs=self.ro_chs,
                     pins=[0], 
                     adc_trig_offset=self.cfg["adc_trig_offset"],
                     wait=True,
                     syncdelay=self.us2cycles(self.cfg["relax_delay"]))
        
        # equivalent to the following:
#         self.trigger(adcs=self.ro_chs,
#                      pins=[0], 
#                      adc_trig_offset=self.cfg["adc_trig_offset"])
#         self.pulse(ch=self.cfg["res_ch"])
#         self.wait_all()
#         self.sync_all(self.us2cycles(self.cfg["relax_delay"]))

### NN scaler setup

### Load an excited state

In [ ]:
config={"res_ch": GEN_CH, #6, # --Fixed
        "ro_chs": [RO_CH], # --Fixed
        "reps": 1, # --Fixed
        "relax_delay": 1.0, # --us
        "res_phase": 0, # --degrees
        "pulse_style": "const", # --Fixed

        "length":100, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length":770, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gain":3000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": 250, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 100, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs": 1
        # Try varying soft_avgs from 1 to 200 averages        
        
       }

In [ ]:
#########################################################
# If you are running Vivado ILA, it is time to arm them #
#########################################################

In [ ]:
from pynq import MMIO

def configure_classifier(scaling_factor, trigger_delay):
    mmio_nn = MMIO(soccfg.NN_0.mmio.base_addr, soccfg.NN_0.mmio.length)
    mmio_nn.write(soccfg.NN_0.register_map.scaling_factor.address, scaling_factor)
    mmio_nn.write(soccfg.NN_0.register_map.trigger_delay.address, trigger_delay)

def reset_classifier(deep_reset = False, index_lo = 0, index_hi = 0):
    RESET_HI = 255
    RESET_LO = 0
    mmio_nn = MMIO(soccfg.NN_0.mmio.base_addr, soccfg.NN_0.mmio.length)
    # This sends a "reset pulse"
    mmio_nn.write(soccfg.NN_0.register_map.out_reset.address, RESET_HI)
    mmio_nn.write(soccfg.NN_0.register_map.out_reset.address, RESET_LO)

    if (deep_reset):
        # Reset in software, in hardware may be more efficient (TBD)
        print('WARNING: This may run for some time... use only for debugging')
        entry_count = ((index_hi - index_lo) + 1) * 2
        for i in range(entry_count):
            soccfg.axi_blk_bram_ctrl_0.mmio.array[i] = 0

def get_classifier_prediction_count():
    mmio_nn = MMIO(soccfg.NN_0.mmio.base_addr, soccfg.NN_0.mmio.length)
    prediction_count = mmio_nn.read(soccfg.NN_0.register_map.out_offset.address)
    return prediction_count

def get_classifier_prediction(index = 0):
    WORD_SIZE_BYTE = 4
    WORD_COUNT_PER_PREDICTION = 2
    mmio_bram = MMIO(soccfg.axi_blk_bram_ctrl_0.base_address, soccfg.axi_blk_bram_ctrl_0.size)
    ground_state_logit = mmio_bram.read(index*WORD_COUNT_PER_PREDICTION*WORD_SIZE_BYTE + 0)
    excited_state_logit = mmio_bram.read(index*WORD_COUNT_PER_PREDICTION*WORD_SIZE_BYTE + WORD_SIZE_BYTE)
    return ground_state_logit, excited_state_logit

def get_classifier_predictions(index_lo = 0, index_hi = 0):
    WORD_SIZE_BYTE = 4
    WORD_COUNT_PER_PREDICTION = 2
    mmio_bram = MMIO(soccfg.axi_blk_bram_ctrl_0.base_address, soccfg.axi_blk_bram_ctrl_0.size)
    # TODO: You can read the whole memory area rather then one element at a time and append
    predictions = []
    for index in range(index_lo, index_hi+1):
        ground_state_logit = mmio_bram.read(index*WORD_COUNT_PER_PREDICTION*WORD_SIZE_BYTE + 0)
        excited_state_logit = mmio_bram.read(index*WORD_COUNT_PER_PREDICTION*WORD_SIZE_BYTE + WORD_SIZE_BYTE)
        predictions.append([ground_state_logit, excited_state_logit])
    return predictions

def print_classifier_buffer(index_lo, index_hi):
    prediction_count = get_classifier_prediction_count()
    buffer_size = len(soccfg.axi_blk_bram_ctrl_0.mmio.array)
    
    print('INFO: prediction count: {:6}'.format(prediction_count))
    print('INFO: buffer size     : {:6}'.format(buffer_size))
    print('INFO:')
    for i in range((index_hi - index_lo) + 1):
        ground_state_logit, excited_state_logit = get_classifier_prediction(i)
        print('INFO: [{:5d}] {:08x} {}'.format(i, ground_state_logit, '<<<' if prediction_count == i else ''))
        print('INFO: [{:5d}] {:08x} {}'.format(i, excited_state_logit, '<<<' if prediction_count == i else ''))

In [ ]:
len(soccfg.axi_blk_bram_ctrl_0.mmio.array)
#8192

In [ ]:
#
# ATTENTION: if the cell fails, please make sure that
#            the base_address matches the associated
#            value in the address editor of the Vivado
#            project.
# 
if HAS_NN:
    from pynq import MMIO
    #
    # Programming the NN
    #
    SCALING_FACTOR = 1
    TRIGGER_DELAY = 1
    
    reset_classifier()
    configure_classifier(SCALING_FACTOR, TRIGGER_DELAY)

In [ ]:
if HAS_NN:
    print_classifier_buffer(index_lo=0, index_hi=10)

In [ ]:
for i in range(1):
    prog = LoopbackProgram(soccfg, config)
    
    iq_list = prog.acquire_decimated(soc, progress=True)
    
    # Plot results.
    for ii, iq in enumerate(iq_list):
        plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]), color='C0')
        plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]), color='C1')
        #plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
        plt.ylabel("a.u.")
        plt.xlabel("Clock ticks")
        plt.title("Averages = " + str(config["soft_avgs"]))
        #plt.legend()
        #plt.show()
    #plt.savefig("images/send_receive_pulse_const.png", dpi=350)
    np.set_printoptions(threshold=np.inf)
    #print([int(element) for pair in zip(*iq_list[0]) for element in pair])
    #import time
    #time.sleep(1)

In [ ]:
import struct
def float_to_hex32(f):
    return format(struct.unpack('!I', struct.pack('!f', f))[0], '08x')

def int_to_twos_complement_hex32(n):
    # If the number is negative, get its two's complement
    if n < 0:
        n = (1 << 32) + n  # "Wrap around" to get 32-bit two's complement
    return format(n, '08x')

# Save trace.
iq = iq_list[0]

# Shows I and Q sequence
iq_sequence = [list(item) for item in zip(*iq)]
iq_sequence = [int(item) for sublist in iq_sequence for item in sublist]

# Float value as hex
#hex_iq_sequence_flt = [float_to_hex32(num) for num in iq_sequence]

# Float -> Int value as hex
#hex_iq_sequence_dec = [int_to_twos_complement_hex32(int(i)) for i in iq_sequence]

# print('Sequence length:', len(iq_sequence))
# print('I (lo, data[15:0])')
# print(iq[0])
# print('Q (hi, data[31:16])')
# print(iq[1])

# print(iq_sequence)

#print(hex_iq_sequence_flt)
#print(hex_iq_sequence_dec)

#for i in range(0,len(iq_sequence),2):
#    print("I {:4.0f} {} {}".format(iq_sequence[i], hex_iq_sequence_flt[i], hex_iq_sequence_dec[i]))
#    print("Q {:4.0f} {} {}".format(iq_sequence[i+1], hex_iq_sequence_flt[i+1], hex_iq_sequence_dec[i+1]))

In [ ]:
#print(iq_sequence[285*2:(285+100)*2])
print(iq_sequence)

In [ ]:
#
# ATTENTION: if the cell fails, please make sure that
#            the base_address matches the associated
#            value in the address editor of the Vivado
#            project.
# 
if HAS_NN:
    from ctypes import *
    def to_float(i):
        cp = pointer(c_int(i))
        fp = cast(cp, POINTER(c_float))
        return fp.contents.value

    ground_state_logit, excited_state_logit = get_classifier_prediction()

    if (ground_state_logit > excited_state_logit):
       print("Prediction: ground state")
    else:
       print("Prediction: excited state")
    print('Logit values as int: [', ground_state_logit, ',', excited_state_logit, ']')
    print('Logit values as hex: [', hex(ground_state_logit), ',', hex(excited_state_logit), ']')
    print('Logit values as flt: [', to_float(ground_state_logit), ',', to_float(excited_state_logit), ']')
else:
    print('No NN in this bitstream!')

In [ ]:
if HAS_NN:
    print_classifier_buffer(index_lo=0, index_hi=63)

In [ ]:
if HAS_NN:
    reset_classifier(deep_reset=True, index_hi=7)

In [ ]:
if HAS_NN:
    print_classifier_buffer(index_lo=0, index_hi=63)

In [ ]:
if HAS_NN:
    print('Readout count: {}'.format(get_classifier_prediction_count()))